<h1> Repeatable splitting </h1>

In this notebook, we will explore the impact of different ways of creating machine learning datasets.

<p>

Repeatability is important in machine learning. If you do the same thing now and 5 minutes from now and get different answers, then it makes experimentation difficult. In other words, you will find it difficult to gauge whether a change you made has resulted in an improvement or not.

In [1]:
import google.datalab.bigquery as bq

<h3> Create a simple machine learning model </h3>

The dataset that we will use is <a href="https://bigquery.cloud.google.com/table/bigquery-samples:airline_ontime_data.flights">a BigQuery public dataset</a> of airline arrival data. Click on the link, and look at the column names. Switch to the Details tab to verify that the number of records is 70 million, and then switch to the Preview tab to look at a few rows.
<p>
We want to predict the arrival delay of an airline based on the departure delay. The model that we will use is a zero-bias linear model:
$$ delay_{arrival} = \alpha * delay_{departure} $$
<p>
To train the model is to estimate a good value for $\alpha$. 
<p>
One approach to estimate alpha is to use this formula:
$$ \alpha = \frac{\sum delay_{departure} delay_{arrival} }{  \sum delay_{departure}^2 } $$
Because we'd like to capture the idea that this relationship is different for flights from New York to Los Angeles vs. flights from Austin to Indianapolis (shorter flight, less busy airports), we'd compute a different $alpha$ for each airport-pair.  For simplicity, we'll do this model only for flights between Denver and Los Angeles.

<h2> Naive random split (not repeatable) </h2>

In [10]:
compute_alpha = """
#standardSQL

   SELECT RAND() AS splitfield,
   arrival_delay,
   departure_delay
FROM
  `bigquery-samples.airline_ontime_data.flights`
WHERE
  departure_airport = 'DEN' AND arrival_airport = 'LAX'

"""

In [12]:
results = bq.Query(compute_alpha).execute().result().to_dataframe()
#prod = results['prod'][0]
#print(prod)
print(results)  # basically you are giving it an index using a random probability distribution 

       splitfield  arrival_delay  departure_delay
0        0.648589           35.0             10.0
1        0.771830           30.0             19.0
2        0.257662           29.0             18.0
3        0.382181           29.0             29.0
4        0.047643           52.0             57.0
5        0.192328           30.0             33.0
6        0.305218           38.0             22.0
7        0.144809           31.0             28.0
8        0.752900           85.0             72.0
9        0.172533           42.0             30.0
10       0.847885           27.0             45.0
11       0.047883           48.0             39.0
12       0.633620           62.0             52.0
13       0.486377           58.0             74.0
14       0.187343          176.0            179.0
15       0.529845           42.0             -3.0
16       0.657522           68.0             80.0
17       0.649828           29.0             14.0
18       0.622404          113.0            111.0


In [8]:
compute_alpha = """
#standardSQL
SELECT 
   arrival_delay * departure_delay AS prod
FROM
(
   SELECT RAND() AS splitfield,
   arrival_delay,
   departure_delay
FROM
  `bigquery-samples.airline_ontime_data.flights`
WHERE
  departure_airport = 'DEN' AND arrival_airport = 'LAX'
)
WHERE
  splitfield < 0.8
"""

In [9]:
results = bq.Query(compute_alpha).execute().result().to_dataframe()
prod = results['prod'][0]
print(prod)
print(results)

216.0
         prod
0       216.0
1        11.0
2       -70.0
3       -32.0
4       460.0
5       -33.0
6       -40.0
7       216.0
8       522.0
9       841.0
10     2964.0
11      -72.0
12      -55.0
13      990.0
14      836.0
15       63.0
16       49.0
17      868.0
18     6120.0
19      208.0
20      273.0
21      -88.0
22      -28.0
23      -78.0
24      -21.0
25     1260.0
26     1215.0
27      176.0
28       99.0
29     1872.0
...       ...
63721    90.0
63722    90.0
63723    90.0
63724    90.0
63725    90.0
63726    90.0
63727    90.0
63728    90.0
63729    90.0
63730    90.0
63731    90.0
63732    90.0
63733    90.0
63734    90.0
63735    90.0
63736    90.0
63737    90.0
63738    90.0
63739    90.0
63740    90.0
63741    90.0
63742    90.0
63743    90.0
63744    90.0
63745    90.0
63746    90.0
63747    90.0
63748    90.0
63749    90.0
63750    90.0

[63751 rows x 1 columns]


In [ ]:
compute_alpha = """
#standardSQL
SELECT 
   SUM(arrival_delay * departure_delay) AS sum
FROM
(
   SELECT RAND() AS splitfield,
   arrival_delay,
   departure_delay
FROM
  `bigquery-samples.airline_ontime_data.flights`
WHERE
  departure_airport = 'DEN' AND arrival_airport = 'LAX'
)
WHERE
  splitfield < 0.8
"""

In [4]:
results = bq.Query(compute_alpha).execute().result().to_dataframe()
sum = results['sum'][0]
print(sum)
print(results)

60048250.0
          sum
0  60048250.0


In [23]:
compute_alpha = """
#standardSQL
SELECT 
   SAFE_DIVIDE(SUM(arrival_delay * departure_delay), SUM(departure_delay * departure_delay)) AS alpha
FROM
(
   SELECT RAND() AS splitfield,
   arrival_delay,
   departure_delay
FROM
  `bigquery-samples.airline_ontime_data.flights`
WHERE
  departure_airport = 'DEN' AND arrival_airport = 'LAX'
)
WHERE
  splitfield < 0.8
"""

In [24]:
results = bq.Query(compute_alpha).execute().result().to_dataframe()
alpha = results['alpha'][0]
print(alpha)

0.9768411624425754


<h3> What is wrong with calculating RMSE on the training and test data as follows? </h3>

In [20]:
compute_rmse = """
#standardSQL
SELECT
  dataset,
  SQRT(AVG((arrival_delay - ALPHA * departure_delay)*(arrival_delay - ALPHA * departure_delay))) AS rmse,
  COUNT(arrival_delay) AS num_flights
FROM (
  SELECT
    IF (RAND() < 0.8, 'train', 'eval') AS dataset,
    arrival_delay,
    departure_delay
  FROM
    `bigquery-samples.airline_ontime_data.flights`
  WHERE
    departure_airport = 'DEN'
    AND arrival_airport = 'LAX' )
GROUP BY
  dataset
"""
bq.Query(compute_rmse.replace('ALPHA', str(alpha))).execute().result()

dataset,rmse,num_flights
eval,12.92969337575481,16097
train,13.122568408000529,63592


In [19]:
compute_rmse = """
#standardSQL

  SELECT
    IF (RAND() < 0.8, 'train', 'eval') AS dataset,
    arrival_delay,
    departure_delay
  FROM
    `bigquery-samples.airline_ontime_data.flights`
  WHERE
    departure_airport = 'DEN'
    AND arrival_airport = 'LAX' 
"""
bq.Query(compute_rmse).execute().result()

dataset,arrival_delay,departure_delay
eval,35.0,10.0
train,30.0,19.0
eval,29.0,18.0
train,29.0,29.0
train,52.0,57.0
train,30.0,33.0
train,38.0,22.0
train,31.0,28.0
train,85.0,72.0
train,42.0,30.0


Hint:
* Are you really getting the same training data in the compute_rmse query as in the compute_alpha query?
* Do you get the same answers each time you rerun the compute_alpha and compute_rmse blocks?

<h3> How do we correctly train and evaluate? </h3>
<br/>
Here's the right way to compute the RMSE using the actual training and held-out (evaluation) data. Note how much harder this feels.

Although the calculations are now correct, the experiment is still not repeatable.

Try running it several times; do you get the same answer?

In [29]:
train_and_eval_rand = """
#standardSQL
WITH
  alldata AS (
  SELECT
    IF (RAND() < 0.8,
      'train',
      'eval') AS dataset,
    arrival_delay,
    departure_delay
  FROM
    `bigquery-samples.airline_ontime_data.flights`
  WHERE
    departure_airport = 'DEN'
    AND arrival_airport = 'LAX' )
SELECT
  dataset,
  COUNT(arrival_delay) AS num_flights
FROM
  alldata
GROUP BY
  dataset
"""

In [30]:
bq.Query(train_and_eval_rand).execute().result()

dataset,num_flights
train,63821
eval,15868


In [ ]:
train_and_eval_rand = """
#standardSQL
WITH
  alldata AS (
  SELECT
    IF (RAND() < 0.8,
      'train',
      'eval') AS dataset,
    arrival_delay,
    departure_delay
  FROM
    `bigquery-samples.airline_ontime_data.flights`
  WHERE
    departure_airport = 'DEN'
    AND arrival_airport = 'LAX' ),
  training AS (
  SELECT
    SAFE_DIVIDE( SUM(arrival_delay * departure_delay) , SUM(departure_delay * departure_delay)) AS alpha
  FROM
    alldata
  WHERE
    dataset = 'train' )
SELECT
  MAX(alpha) AS alpha,
  dataset,
  SQRT(AVG((arrival_delay - alpha * departure_delay)*(arrival_delay - alpha * departure_delay))) AS rmse,
  COUNT(arrival_delay) AS num_flights
FROM
  alldata,
  training
GROUP BY
  dataset
"""

In [26]:
bq.Query(train_and_eval_rand).execute().result()

alpha,dataset,rmse,num_flights
0.9744019005441548,train,13.05061519453309,63980
0.9744019005441548,eval,13.218310591164133,15709


<h2> Using HASH of date to split the data </h2>

Let's split by date and train.

In [7]:
compute_alpha = """
#standardSQL
SELECT 
   SAFE_DIVIDE(SUM(arrival_delay * departure_delay), SUM(departure_delay * departure_delay)) AS alpha
FROM
  `bigquery-samples.airline_ontime_data.flights`
WHERE
  departure_airport = 'DEN' AND arrival_airport = 'LAX'
  AND MOD(ABS(FARM_FINGERPRINT(date)), 10) < 8
"""
results = bq.Query(compute_alpha).execute().result().to_dataframe()
alpha = results['alpha'][0]
print(alpha)

0.9758039143620403


We can now use the alpha to compute RMSE. Because the alpha value is repeatable, we don't need to worry that the alpha in the compute_rmse will be different from the alpha computed in the compute_alpha.

In [8]:
compute_rmse = """
#standardSQL
SELECT
  IF(MOD(ABS(FARM_FINGERPRINT(date)), 10) < 8, 'train', 'eval') AS dataset,
  SQRT(AVG((arrival_delay - ALPHA * departure_delay)*(arrival_delay - ALPHA * departure_delay))) AS rmse,
  COUNT(arrival_delay) AS num_flights
FROM
    `bigquery-samples.airline_ontime_data.flights`
WHERE
    departure_airport = 'DEN'
    AND arrival_airport = 'LAX'
GROUP BY
  dataset
"""
print(bq.Query(compute_rmse.replace('ALPHA', str(alpha))).execute().result().to_dataframe().head())

  dataset       rmse  num_flights
0    eval  12.764685        15671
1   train  13.160712        64018


Note also that the RMSE on the evaluation dataset more from the RMSE on the training dataset when we do the split correctly.  This should be expected; in the RAND() case, there was leakage between training and evaluation datasets, because there is high correlation between flights on the same day.
<p>
This is one of the biggest dangers with doing machine learning splits the wrong way -- <b> you will develop a false sense of confidence in how good your model is! </b>

Copyright 2018 Google Inc.
Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at
http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.